In [1]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
import numpy as np

from rdkit.Chem import MACCSkeys
import pickle
from exmol_our import exmol
from functools import partial

from sklearn.ensemble import RandomForestRegressor
import pickle
import pandas as pd
from sklearn.model_selection import KFold
import random

import warnings
warnings.filterwarnings('ignore')
import json
import pickle
import scipy.cluster.hierarchy as hc

from rdkit import Chem  
from rdkit.Chem import MolFromSmiles as smi2mol 

from rdkit.Chem import AllChem  
from rdkit.DataStructs.cDataStructs import BulkTanimotoSimilarity, TanimotoSimilarity 
from rdkit.Chem import Mol  

import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
data_type = 'caco'
feature_type = 'maccs'
model_type = 'hist'

## Load fingerprint data

In [3]:
data_path = f'../../data/processed/{data_type}_{feature_type}_all.csv'
df = pd.read_csv(data_path)

X_fp = df.loc[:, df.columns != df.columns[0]]
y_fp = df[df.columns[0]]

X_fp = np.array(X_fp)
y_fp = np.array(y_fp)

## Get predictions

In [4]:
first_splits = KFold(n_splits=5, shuffle=True, random_state=42)
predictions = np.zeros((len(X_fp)))
for fold_1, (train_idx,test_idx) in enumerate(first_splits.split(np.arange(len(X_fp)))):
    train_x = X_fp[train_idx]
    test_x = X_fp[test_idx]
    model = pickle.load(open(f'../../models/{data_type}_{feature_type}_{model_type}_{fold_1+1}.pkl', 'rb'))
    predictions[test_idx] = model.predict(test_x)

## Load smiles data

In [5]:
data_path = f'../../data/processed/{data_type}_smiles_all.csv'
df = pd.read_csv(data_path)
df['predictions'] = predictions
df = df[df['predictions']<=10] # get inactive examples

# 2. Split data to X and y 
X_sm = df[['smiles']]
y_sm = df[['permeability']]

## Clustering

In [7]:
ECFP4 = [AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles[0]), 2) for smiles in X_sm.values]
M = np.array([BulkTanimotoSimilarity(f, ECFP4) for f in ECFP4])
M2 = 1 - M
dist_df = pd.DataFrame(M2, index = X_sm.values, columns= X_sm.values)
clustered = hc.linkage(M2, method='complete')

In [8]:
from scipy.cluster.hierarchy import cut_tree
n_clusters = 20
clusters = cut_tree(clustered, n_clusters = n_clusters).T
df_clustered = pd.DataFrame({'Column1': list(X_sm.values), 'Column2': clusters[0]})

In [9]:
from collections import Counter
c = Counter(list(df_clustered['Column2']))
sorted(c.items(), key=lambda i: i[1])

[(17, 16),
 (2, 26),
 (19, 29),
 (16, 37),
 (14, 41),
 (10, 43),
 (8, 48),
 (9, 48),
 (18, 49),
 (12, 51),
 (15, 51),
 (5, 72),
 (13, 73),
 (11, 96),
 (7, 117),
 (6, 137),
 (1, 162),
 (4, 171),
 (0, 269),
 (3, 304)]

## Get samples

In [10]:
random.seed(42)
np.random.seed(42)
n_clusters = 100
x = 1000/len(X_sm.values)
samples = []
for i in range(n_clusters):
    C_i = np.where(df_clustered.Column2 == i)[0].tolist() 
    n_i = len(C_i)
    sample_i = random.sample(C_i, round(x * n_i)) 
    samples += list(sample_i)
print(len(samples))

1000


## Exmol

In [11]:
def GetMACCS(smiles):
    """Function to convert smiles to numpy array of MACCS keys"""
    mol = Chem.MolFromSmiles(smiles)
    fp = MACCSkeys.GenMACCSKeys(mol)
    fp.ToBitString() 
    fp_list = list(fp)[1:]
    return np.array(fp_list)

In [12]:
def model_predict(smiles, model):
    fingerprint = GetMACCS(smiles)
    prediction  = model.predict(fingerprint.reshape(1,-1))
    return prediction

In [13]:
def get_changed_idxs(cfs):
    orignal_smiles = cfs[0].smiles
    orignal_fs = GetMACCS(orignal_smiles)
    counterfc_fp = [GetMACCS(counterfc.smiles) for counterfc in cfs[1:] if counterfc.label.startswith('Increase')]
    return [(np.where((orignal_fs-cfp)==1)[0], np.where((orignal_fs-cfp)==-1)[0]) for cfp in counterfc_fp] # (-,+)

In [14]:
def get_cfs(examples, model, thd):
    changed_indx_list = []
    for exp in examples.values:
        space = exmol.sample_space(exp[0], partial(model_predict, model=model), preset="medium", num_samples=500, batched=False)
        prediction = model_predict(exp[0], model)[0]
        if prediction<0:
            radius = round(float(thd+abs(prediction)),2)
        else:
            radius = round(float(thd-prediction),2)
        cfs = exmol.rcf_explain(space, radius, nmols=4)
        changed_idxs = get_changed_idxs(cfs)
        changed_indx_list += changed_idxs
    return changed_indx_list 

In [15]:
first_splits = KFold(n_splits=5, shuffle=True, random_state=42)
changed_indx_list = []
for fold_1, (train_idx,test_idx) in enumerate(first_splits.split(np.arange(len(X_sm)))):
    test_idx = np.array(list(set(test_idx) & set(samples)))
    test_x = X_sm.iloc[test_idx]
    model = pickle.load(open(f'../../models/{data_type}_{feature_type}_{model_type}_{fold_1+1}.pkl', 'rb'))
    changed_indx_list += get_cfs(test_x, model, 10.0)

🤘Done🤘: 100%|██████████| 121.0/121 [00:00<00:00, 1384.30it/s]                    
🤘Done🤘: 100%|██████████| 96.0/96 [00:00<00:00, 1712.13it/s]                      
🤘Done🤘: 100%|██████████| 68.0/68 [00:00<00:00, 1528.45it/s]                      
🤘Done🤘: 100%|██████████| 157.0/157 [00:00<00:00, 1296.35it/s]                    
🤘Done🤘: 100%|██████████| 182.0/182 [00:00<00:00, 1510.20it/s]                    
🤘Done🤘: 100%|██████████| 112.0/112 [00:00<00:00, 1777.86it/s]                    
🤘Done🤘: 100%|██████████| 160.0/160 [00:00<00:00, 1363.41it/s]                    
🤘Done🤘: 100%|██████████| 147.0/147 [00:00<00:00, 902.81it/s]                     
🤘Done🤘: 100%|██████████| 124.0/124 [00:00<00:00, 1294.03it/s]                    
🤘Done🤘: 100%|██████████| 160.0/160 [00:00<00:00, 1184.37it/s]                    
🤘Done🤘: 100%|██████████| 300.0/300 [00:00<00:00, 628.00it/s]                     
🤘Done🤘: 100%|██████████| 149.0/149 [00:00<00:00, 1484.83it/s]                    
🤘Done🤘: 100%|███

In [16]:
len(changed_indx_list)

1688

#### Mean removed fingerprints

In [17]:
np.mean([len(list(c[0])) for c in changed_indx_list])

3.588270142180095

#### Mean added fingerprints

In [18]:
np.mean([len(list(c[1])) for c in changed_indx_list])

4.143364928909953

#### Most frequent removed fingerprints

In [19]:
with open('../../maccs_keys_dict.pickle', 'rb') as fp:
    Maccs_keys_dict = pickle.load(fp)

In [20]:
def map_to_smarts(df):
    df_mapped = df + 1
    df_mapped = df_mapped.applymap(lambda x: Maccs_keys_dict[x][0])
    return df_mapped

In [21]:
from collections import Counter
counts = Counter([item for sublist in [list(c[0]) for c in changed_indx_list] for item in sublist])
print(len(counts))
sorted_counts = sorted(counts.items(), key=lambda i: i[1], reverse=True)[:10]
df_sorted_counts = pd.DataFrame({'Smarts': [x[0] for x in sorted_counts], 'Counts': [x[1] for x in sorted_counts]})
df_sorted_counts

141


,Smarts,Counts
0,130,171
1,139,170
2,89,156
3,91,155
4,52,141
5,103,125
6,109,121
7,138,117
8,90,113
9,145,112


In [22]:
df_sorted_counts['Smarts'] = map_to_smarts(df_sorted_counts[['Smarts']])
df_sorted_counts.to_csv(f'files/{data_type}/{data_type}_{feature_type}_{model_type}_most_frequent_removed_fps.csv', index=False)
df_sorted_counts

,Smarts,Counts
0,[!#6;!#1;!H0],171
1,[#8],170
2,"[$([!#6;!#1;!H0]~*~*~[CH2]~*),$([!#6;!#1;!H0;R...",156
3,[#8]~[#6](~[#7])~[#6],155
4,[!#6;!#1;!H0]~*~*~*~[!#6;!#1;!H0],141
5,[!#6;!#1;!H0]~*~[CH2]~*,125
6,[#7]~[#6]~[#8],121
7,[O;!H0],117
8,"[$([!#6;!#1;!H0]~*~*~*~[CH2]~*),$([!#6;!#1;!H0...",113
9,[#8],112


#### Most frequent added fingerprints

In [23]:
counts = Counter([item for sublist in [list(c[1]) for c in changed_indx_list] for item in sublist])
print(len(counts))
sorted_counts = sorted(counts.items(), key=lambda i: i[1], reverse=True)[:10]
df_sorted_counts = pd.DataFrame({'Smarts': [x[0] for x in sorted_counts], 'Counts': [x[1] for x in sorted_counts]})
df_sorted_counts

144


,Smarts,Counts
0,75,227
1,98,223
2,123,202
3,51,195
4,93,192
5,68,180
6,44,162
7,96,125
8,118,125
9,10,124


In [24]:
df_sorted_counts['Smarts'] = map_to_smarts(df_sorted_counts[['Smarts']])
df_sorted_counts.to_csv(f'files/{data_type}/{data_type}_{feature_type}_{model_type}_most_frequent_added_fps.csv', index=False)
df_sorted_counts

,Smarts,Counts
0,[#6]=[#6](~*)~*,227
1,[#6]=[#6],223
2,[!#6;!#1]~[!#6;!#1],202
3,[#7]~[#7],195
4,[!#6;!#1]~[#7],192
5,[!#6;!#1]~[!#6;!#1;!H0],180
6,[#6]=[#6]~[#7],162
7,[#7]~*~*~*~[#8],125
8,[#7]=*,125
9,*1~*~*~*~1,124
